**Clustering Toronto neighborhoods based on their characteristics**

In [3]:
# import libraries

import pandas as pd
import numpy as np
!pip install geocoder
import geocoder
import requests
import folium as fl
from sklearn.cluster import KMeans

     |████████████████████████████████| 98 kB 3.9 MB/s eta 0:00:01
You should consider upgrading via the '/opt/anaconda3/bin/python3 -m pip install --upgrade pip' command.


In [5]:
# scrape Toronto neighborhood info from Wikipedia

neighborhoodswiki = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"

dfHoods = pd.read_html(neighborhoodswiki,flavor="bs4")[0]
dfHoods.head()

,0,1,2,3,4,5,6,7,8
0,M1ANot assigned,M2ANot assigned,M3ANorth York(Parkwoods),M4ANorth York(Victoria Village),M5ADowntown Toronto(Regent Park / Harbourfront),M6ANorth York(Lawrence Manor / Lawrence Heights),M7AQueen's Park(Ontario Provincial Government),M8ANot assigned,M9AEtobicoke(Islington Avenue)
1,M1BScarborough(Malvern / Rouge),M2BNot assigned,M3BNorth York(Don Mills)North,M4BEast York(Parkview Hill / Woodbine Gardens),"M5BDowntown Toronto(Garden District, Ryerson)",M6BNorth York(Glencairn),M7BNot assigned,M8BNot assigned,M9BEtobicoke(West Deane Park / Princess Garden...
2,M1CScarborough(Rouge Hill / Port Union / Highl...,M2CNot assigned,M3CNorth York(Don Mills)South(Flemingdon Park),M4CEast York(Woodbine Heights),M5CDowntown Toronto(St. James Town),M6CYork(Humewood-Cedarvale),M7CNot assigned,M8CNot assigned,M9CEtobicoke(Eringate / Bloordale Gardens / Ol...
3,M1EScarborough(Guildwood / Morningside / West ...,M2ENot assigned,M3ENot assigned,M4EEast Toronto(The Beaches),M5EDowntown Toronto(Berczy Park),M6EYork(Caledonia-Fairbanks),M7ENot assigned,M8ENot assigned,M9ENot assigned
4,M1GScarborough(Woburn),M2GNot assigned,M3GNot assigned,M4GEast York(Leaside),M5GDowntown Toronto(Central Bay Street),M6GDowntown Toronto(Christie),M7GNot assigned,M8GNot assigned,M9GNot assigned


In [54]:
# this table is not at all what the assignment says it is. I'm not great with BeautifulSoup so I'm using this processing nightmare to make it work

codes = []

for i in range(0,9):
    for row in dfHoods[i]:
        codes.append(row[0:3])

boroughs = []

for i in range(0,9):
    for row in dfHoods[i]:
        end = row.find("(")
        if end != -1:
            boroughs.append(row[3:end])
        else:
            boroughs.append(row[3:])

neighborhoods = []

for i in range(0,9):
    for row in dfHoods[i]:
        start = row.find("(")+1
        end = row.find(")")
        if start != -1 and end != -1:
            neighborhoods.append(row[start:end])
        elif start != - 1 and end == -1:
            neighborhoods.append(row[start:len(row)-1])
        else:
            neighborhoods.append(row[3:])

**I used this method because the course suggested using pandas to scrape the html table on the page. The table is not formatted in the way pandas needs (see dfHoods example above) so I used loops and string slicing to get the information I needed.** 

In [120]:
# now to create the df and clean it up

dfClean = pd.DataFrame(boroughs,codes)
dfClean["Neighborhood"] = neighborhoods
dfClean.reset_index(inplace=True)
dfClean.rename(columns={"index":"PostalCode",0:"Borough"},inplace=True)
dfClean = dfClean[dfClean["Borough"]!="Not assigned"]
dfClean["Neighborhood"] = dfClean["Neighborhood"].str.replace(" / ",", ")
dfClean.reset_index(inplace=True,drop=True)

**I created and cleaned the df to match the format required in the course. It's in order based on the table order**

In [124]:
dfClean.head()

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [125]:
dfClean.shape

(103, 3)